# Installing Python and Gurobi

## Python and PIP
https://realpython.com/installing-python/

## Gurobi
* Register in Gurobi as an Academic user: https://www.gurobi.com
* Get a free academic license: https://support.gurobi.com/hc/en-us/articles/360040541251-How-do-I-obtain-a-free-academic-license
    * Note: To download the license and install it you need to be on the campus network. Due to some tunnelling issues with the Cisco VPN, it seems that using UWaterloo VPN will not work, so I believe you can only do this if you are on campus.
* Install Gurobi.
    * Windows: https://www.youtube.com/watch?v=z7t0p5J9YcQ
    * Linux: https://www.youtube.com/watch?v=OYuOKXPJ5PI
    * MacOS: https://www.youtube.com/watch?v=dcFstZl5Va4
* Install Gurobi Python API: https://support.gurobi.com/hc/en-us/articles/360044290292-How-do-I-install-Gurobi-for-Python
    * ```
      python -m pip install gurobipy
      ```

# Python types (strings, integers and floats)
Before we write at a Gurobi example, let us just take a look at the following snippets.

In [1]:
# String, integer and float variables
stringVar = "123"
integerVar = 123
floatVar = 123.0
print(type(stringVar))
print(type(integerVar))
print(type(floatVar))

<class 'str'>
<class 'int'>
<class 'float'>


In [2]:
# Adding strings is equivalent to concatenating them
print(stringVar + "123")

123123


In [3]:
# Adding floats (or integers) is as you expect
print(floatVar + 123.0)

246.0


In [4]:
# But you cannot add strings and floats!
print(stringVar + 123.0)

TypeError: can only concatenate str (not "float") to str

In [5]:
# We can easily convert between these different types using the functions `str`, `int` and `float`
print(float(stringVar) + 123.0)
print(stringVar + str(123.0))

246.0
123123.0


# Simple Python+Gurobi example

We will now use Gurobi Python API to solve the following simple integer linear program.
$$
\begin{align}
\text{max} &~~2 x_1 + 4 x_2 \\
\text{s.t.} &~~3 x_1 + 2 x_2 \leq 6, \\
&~~x_1 + 5 x_2 \leq 5, \\
&~~x_1 \in \mathbb{Z}, \\
&~~x_1 \geq 0, \\
&~~x_2 \geq 0.
\end{align}
$$

In [6]:
# First we import Gurobi library and create a Gurobi model.
import gurobipy as gp
from gurobipy import GRB
m = gp.Model("basic")

# `m` is an object of the type Gurobi model.
print(type(m))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-16
<class 'gurobipy.Model'>


In [7]:
# Next, we create the variables.
# NOTE: By default all variables have LB = 0, UB = infty (float('inf'))
# and are of type GRB.CONTINUOUS. You may change these parameters (they are optional)
x1 = m.addVar( lb = 0.0, ub = float('inf') , vtype = GRB.INTEGER, name = "x1" )
x2 = m.addVar( vtype = GRB.CONTINUOUS, name = "x2"  )

# x1 and x2 are objects of the type Gurobi variable.
print(type(x1))
print(type(x2))

<class 'gurobipy.Var'>
<class 'gurobipy.Var'>


In [8]:
# Set the objective function to maximize 2 * x1 + 4 * x2.
m.setObjective( 2 * x1 + 4 * x2, GRB.MAXIMIZE )

In [9]:
# Add the two constraints in the model.
m.addConstr( 3 * x1 + 2 * x2 <= 6, "c1")
m.addConstr( x1 + 5 * x2 <= 5, "c2" )

<gurobi.Constr *Awaiting Model Update*>

In [10]:
# Solve the model!
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i7-4720HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x167672fe
Variable types: 1 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [2e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 6e+00]
Found heuristic solution: objective 4.0000000
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)

Root relaxation: objective 5.200000e+00, 0 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       5.2000000    5.20000  0.00%     -    0s

Explore

In [11]:
# Check the optimization status (see https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html).
print(m.status == GRB.OPTIMAL)

True


In [12]:
# Since the status is `GRB.OPTIMAL`, we can query the optimal solution found by Gurobi.
# To get the optimal objective value we use the attribute `objVal` of the Gurobi model.
print("Optimal cost: " + str(m.objVal))

Optimal cost: 5.2


In [13]:
# To get the optimal variable values found by Gurobi, we use the attribute `X` of the variables.
print("x1: " + str(x1.X))
print("x2: " + str(x2.X))

x1: 1.0
x2: 0.8


In [14]:
# And we can check that indeed, this solution matches the objective value we found with `m.objVal`.
print(2.0 * x1.X + 4.0 * x2.X)

5.2


# Exercise 1
Write a Gurobi Python script that solves the following integer programming model.
$$
\begin{align}
\text{max} &~~10 x_1 + 5 x_2 + 3 x_3 \\
\text{s.t.} &~~x_1 + x_2 + x_3 \leq 1, \\
&~~x_1, x_2, x_3 \in \mathbb{Z}, \\
&~~x_1, x_2, x_3 \geq 0.
\end{align}
$$
Note that the optimal solution to this problem is $x_1 = 1, x_2 = 0, x_3 = 0$ with objective value 10. You should check that your script returns this solution.

# Exercise 2
Write a Gurobi Python script that solves the following linear programming model.
$$
\begin{align}
\text{max} &~~2 x_1 + 4 x_2 \\
\text{s.t.} &~~3 x_1 + 2 x_2 \leq 6, \\
&~~6 x_1 + 4 x_2 \geq 13, \\
&~~x_1 \geq 0, \\
&~~x_2 \geq 0.
\end{align}
$$
By carefully looking at the constraints, we can see that this problem is infeasible. Check in the Gurobi documentation (https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html) what is the "Status Code" for infeasible models and confirm that your script also returns this same Status Code. 